In [1]:
import uuid
import re
import os
import subprocess
from subprocess import CalledProcessError
from datetime import datetime

def linux_cmd(cmd):
    return subprocess.check_output(cmd, shell=True).decode()
# linux_cmd("echo 'hello'")

def vms2dict():
    cmd="VBoxManage list vms"
    result = linux_cmd(cmd)
    res = list(map(lambda x: x.split(), result.split('\n')[:-1]))
    cut = lambda x:x[1:-1]
    res = dict([list(map(lambda x:cut(x), i)) for i in res])
    return res

def vm_info_dict(name):
    cmd="VBoxManage showvminfo {} --details".format(name)
    result = linux_cmd(cmd).split('\n')
    result = list(map(lambda x:x.split(':'),result))
    result = [list(map(lambda x: x.strip(), i)) for i in result]
    result = list(filter(lambda x:len(x)==2, result))
    result = list(filter(lambda x:type(x)==list, result))
    result = dict(result)
    return result

def get_vm_loc(machine_name):
    cpath = vm_info_dict(machine_name)["Config file"]
    vm_loc = os.path.split(cpath)[0]
    return vm_loc

yes = {'yes','y'}
no = {'no','n'}

def identical_clone(machine_name):
    try:
        machineinfo = vm_info_dict(machine_name)
    except CalledProcessError:
        print('{} not found'.format(machine_name))
    
    current_hardware_uuid = machineinfo['Hardware UUID']
    new_hardware_uuid = str(uuid.uuid4())
    
    prompt_str = "Sure want to change {} Hardware UUID?".format(machine_name)
    choice = input(prompt=prompt_str).lower()
    if choice in yes:
        now = '{}'.format(datetime.now())
        chng_log = '{}: {} hardware uuid was changed from {} to {}\n'.format(now, machine_name,
                                                                             current_hardware_uuid, new_hardware_uuid)
        chng_log_path = os.path.join(get_vm_loc(machine_name), 'hardware_uuid_change_log.txt')
        f = open(chng_log_path, 'a+')
        f.write(chng_log)
        f.close()
        cmd = 'VBoxManage modifyvm {} --hardwareuuid {}'.format(machineinfo['UUID'], new_hardware_uuid)
        linux_cmd(cmd)
        print('hardware uuid has been changed to {}'.format(new_hardware_uuid))
    if choice in no:
        print('operation aborted')
        pass
    else:
        pass
    return

identical_clone('win10pro_v1')

Sure want to change win10pro_v1 Hardware UUID?y
hardware uuid has been changed to ce3e10b2-8957-4d5d-941a-e45779a31054
